<a href="https://colab.research.google.com/github/zar8998/All-Projects/blob/main/Code/GTN_MultiOmics_PPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank" href="https://colab.research.google.com/github/avakanski/Graph_NNs_for_Multi_Omics_Integration/blob/main/Code/GTN_MultiOmics_PPI.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.2 MB/s eta 0:00:00


In [ ]:
# GTN using multi-omics data (mRNA, miRNA and DNA methylation) with PPI graph structure (5 fold cross validation)
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import TransformerConv
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import datetime
now = datetime.datetime.now

# Check if GPU is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Step 1: Load the PPI data
ppi_file_path = 'drive/My Drive/Projects/Gene_Expression_Project/PPI.csv'
ppi_df = pd.read_csv(ppi_file_path)

# Step 2: Concatenate 'stringId_A' and 'stringId_B' to calculate the number of connections (degree)
all_proteins = pd.concat([ppi_df['stringId_A'], ppi_df['stringId_B']])

# Step 3: Count the number of connections for each protein
protein_connections = all_proteins.value_counts()

# Step 4: Define a degree threshold to select only highly connected proteins (e.g., 200 or more connections)
degree_threshold = 200
high_degree_proteins = protein_connections[protein_connections >= degree_threshold].index

# Step 5: Filter the PPI data to include only edges where both proteins have a high number of connections
ppi_filtered = ppi_df[
    ppi_df['stringId_A'].isin(high_degree_proteins) &
    ppi_df['stringId_B'].isin(high_degree_proteins)
]

# Step 6: Map the high-degree proteins to unique node IDs
proteins = pd.concat([ppi_filtered['stringId_A'], ppi_filtered['stringId_B']]).unique()
protein_to_id = {protein: idx for idx, protein in enumerate(proteins)}

# Step 7: Create edge index (this will be the input for GTN)
edges = ppi_filtered[['stringId_A', 'stringId_B']].map(lambda x: protein_to_id[x])
edge_index = torch.tensor(edges.values.T, dtype=torch.long).to(device)

In [ ]:
# Step 8: Load and preprocess the multi-omics data
!wget https://www.idahofallshighered.org/vakanski/Codes_Data/mRNA_miRNA_Meth_integrated.csv
file_path = 'mRNA_miRNA_Meth_integrated.csv'
df = pd.read_csv(file_path)
df.drop(df.columns[0], axis=1, inplace=True)
Y = df.iloc[:, -1].copy()

# Remove non-numeric columns
df = df.select_dtypes(include=[np.number])
X = df.values

num_classes = len(set(Y))
print("Number of classes:", num_classes)
num_samples = X.shape[0]
print("Number of samples:", num_samples)
num_Features = X.shape[1]
print("Number of Features:", num_Features)

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

# Convert data to PyTorch tensors and move to device
X = torch.tensor(X, dtype=torch.float).to(device)
Y = torch.tensor(Y, dtype=torch.long).to(device)

--2025-03-07 18:48:58--  https://www.webpages.uidaho.edu/vakanski/Codes_Data/mRNA_miRNA_Meth_integrated.csv
Resolving www.webpages.uidaho.edu (www.webpages.uidaho.edu)... 129.101.105.230
Connecting to www.webpages.uidaho.edu (www.webpages.uidaho.edu)|129.101.105.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123599052 (118M) [application/octet-stream]
Saving to: ‘mRNA_miRNA_Meth_integrated.csv’

mRNA_miRNA_Meth_int 100%[===================>] 117.87M  37.8MB/s    in 3.7s    

2025-03-07 18:49:03 (31.7 MB/s) - ‘mRNA_miRNA_Meth_integrated.csv’ saved [123599052/123599052]

Number of classes: 32
Number of samples: 8464
Number of Features: 2793


In [ ]:
# Step 9: Create PyTorch Geometric data object using the edge_index from the filtered PPI network
data = Data(x=X, edge_index=edge_index)

# Step 10: Define the GTN model
class GTN(nn.Module):
    def __init__(self, num_features, num_classes, hidden_feats=1024, num_layers=2, dropout=0.5):
        super(GTN, self).__init__()
        self.convs = nn.ModuleList()
        self.convs.append(TransformerConv(num_features, hidden_feats, heads=1, dropout=dropout))
        for _ in range(num_layers - 1):
            self.convs.append(TransformerConv(hidden_feats, hidden_feats, heads=1, dropout=dropout))
        self.fc = nn.Linear(hidden_feats, num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = self.dropout(x)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

# Step 11: Set up K-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True)

# Initialize lists to store metrics for each fold
precision_scores = []
recall_scores = []
accuracy_scores = []
F1Measure = []

# Set hyperparameters
hidden_feats = 1024
num_layers = 2
dropout = 0.5
lr = 0.001
weight_decay = 0
num_epochs = 100

In [ ]:
# Step 12: Training and Evaluation
t = now()
for train_index, test_index in kf.split(X.cpu()):
    # Split the data into training and testing sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    # Create train/test data using the same PPI edge_index
    train_data = Data(x=X_train, edge_index=edge_index)
    test_data = Data(x=X_test, edge_index=edge_index)

    # Create the GTN model
    model = GTN(
        num_features=X.shape[1],
        num_classes=len(set(Y.cpu().numpy())),
        hidden_feats=hidden_feats,
        num_layers=num_layers,
        dropout=dropout
    ).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=10)
    criterion = nn.NLLLoss()

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        out = model(train_data)
        loss = criterion(out, y_train)
        loss.backward()
        optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            logits = model(test_data)
            pred = torch.argmax(logits, dim=1)
            acc = accuracy_score(y_test.cpu().numpy(), pred.cpu().numpy())
            # print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Accuracy: {acc:.4f}')
            scheduler.step(acc)

    # Testing
    model.eval()
    with torch.no_grad():
        logits = model(test_data)
        pred = torch.argmax(logits, dim=1)
        test_acc = accuracy_score(y_test.cpu().numpy(), pred.cpu().numpy())
        precision = precision_score(y_test.cpu().numpy(), pred.cpu().numpy(), average='macro', zero_division=1)
        recall = recall_score(y_test.cpu().numpy(), pred.cpu().numpy(), average='macro')
        f1 = f1_score(y_test.cpu().numpy(), pred.cpu().numpy(), average='macro')

        accuracy_scores.append(test_acc)
        precision_scores.append(precision)
        recall_scores.append(recall)
        F1Measure.append(f1)
print('Training time: %s' % (now() - t))

Training time: 0:01:12.276266


In [ ]:
# Calculate the average metrics across all folds
average_accuracy = np.mean(accuracy_scores)
average_precision = np.mean(precision_scores)
average_recall = np.mean(recall_scores)
average_f1 = np.mean(F1Measure)

print("Average accuracy =", average_accuracy)
print("Accuracy std sev =", np.std(accuracy_scores))
print("Average precision =", average_precision)
print("Precision std sev =", np.std(precision_scores))
print("Average recall =", average_recall)
print("Recall std sev =", np.std(recall_scores))
print("Average F1 score =", average_f1)
print("F1 std dev =", np.std(F1Measure))

Average accuracy = 0.9520330550354051
Accuracy std sev = 0.004433865609198085
Average precision = 0.9446092787639447
Precision std sev = 0.007422989624697369
Average recall = 0.9209231654365416
Recall std sev = 0.013145008384676457
Average F1 score = 0.9230865539887884
F1 std dev = 0.012157202620385159


In [ ]:
# GCN using multi-omics data (mRNA, miRNA, DNA methylation) with correlation matrix graph structure (5-fold cross-validation)
# Optimized for Google Colab with manual file upload and CSV error handling

# Install required libraries
!pip install -q torch-geometric

# Import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GraphConv
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import datetime
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

# Check if GPU is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Prompt user to upload the dataset
from google.colab import files
print("Please upload 'mRNA_miRNA_Meth_integrated.csv'.")
uploaded = files.upload()

# Load CSV with error handling
file_name = list(uploaded.keys())[0]
try:
    # Attempt to load with different delimiters and inspect the file
    df = pd.read_csv(file_name, sep=',', engine='python', on_bad_lines='warn')
    print(f"Successfully loaded CSV. Shape: {df.shape}")
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")
    print("Trying with alternative delimiter (e.g., tab or semicolon)...")
    try:
        df = pd.read_csv(file_name, sep='\t', engine='python', on_bad_lines='warn')
        print(f"Loaded with tab delimiter. Shape: {df.shape}")
    except:
        try:
            df = pd.read_csv(file_name, sep=';', engine='python', on_bad_lines='warn')
            print(f"Loaded with semicolon delimiter. Shape: {df.shape}")
        except Exception as e2:
            print(f"Failed to load CSV: {e2}")
            print("Please inspect the CSV file for issues (e.g., delimiter, missing values, or row length mismatches).")
            raise

# Inspect the first few rows and columns
print("\nFirst 5 rows of the dataset:")
print(df.head())
print(f"\nNumber of columns: {df.shape[1]}")

# Preprocess data
try:
    df.drop(df.columns[0], axis=1, inplace=True)  # Drop first column (assumed index)
    Y = df.iloc[:, -1].copy()  # Extract labels
except IndexError as e:
    print(f"Error accessing columns: {e}")
    print("Check if the CSV has at least 2 columns (features + label).")
    raise

# Remove non-numeric columns
df = df.select_dtypes(include=[np.number])
X = df.values

# Print dataset details
num_classes = len(set(Y))
num_samples = X.shape[0]
num_features = X.shape[1]
print(f"Number of classes: {num_classes}")
print(f"Number of samples: {num_samples}")
print(f"Number of features: {num_features}")

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

# Define the GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_feats, num_classes, num_layers=3, dropout=0.3):
        super(GCN, self).__init__()
        self.conv_layers = nn.ModuleList([GraphConv(in_feats, hidden_feats)])
        self.conv_layers.extend([GraphConv(hidden_feats, hidden_feats) for _ in range(num_layers - 1)])
        self.final_conv = GraphConv(hidden_feats, num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for conv in self.conv_layers:
            x = torch.relu(conv(x, edge_index))
            x = self.dropout(x)
        x = self.final_conv(x, edge_index)
        return x

# Set up 5-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Initialize lists to store metrics
precision_scores = []
recall_scores = []
accuracy_scores = []
f1_scores = []

# Training and evaluation
start_time = datetime.datetime.now()
print(f"Training started at: {start_time}")

for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
    print(f"\nFold {fold}/{k}")

    # Split data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    # Convert to tensors
    X_train = torch.FloatTensor(X_train).to(device)
    y_train = torch.LongTensor(y_train).to(device)
    X_test = torch.FloatTensor(X_test).to(device)
    y_test = torch.LongTensor(y_test).to(device)

    # Calculate correlation matrices and create edge indices
    correlation_matrix_train = np.corrcoef(X_train.cpu().numpy(), rowvar=False)
    correlation_matrix_test = np.corrcoef(X_test.cpu().numpy(), rowvar=False)

    # Handle potential NaN values
    correlation_matrix_train = np.nan_to_num(correlation_matrix_train, nan=0.0)
    correlation_matrix_test = np.nan_to_num(correlation_matrix_test, nan=0.0)

    # Create edge indices based on correlation threshold (0.9)
    src_train, dst_train = np.where(correlation_matrix_train > 0.9)
    src_test, dst_test = np.where(correlation_matrix_test > 0.9)

    edge_index_train = torch.tensor([src_train, dst_train], dtype=torch.long).to(device)
    edge_index_test = torch.tensor([src_test, dst_test], dtype=torch.long).to(device)

    # Create PyTorch Geometric data objects
    train_data = Data(x=X_train, edge_index=edge_index_train).to(device)
    test_data = Data(x=X_test, edge_index=edge_index_test).to(device)

    # Initialize model, loss function, and optimizer
    model = GCN(in_feats=num_features, hidden_feats=512, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        outputs = model(train_data)
        loss = criterion(outputs, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print progress every 20 epochs
        if (epoch + 1) % 20 == 0:
            model.eval()
            with torch.no_grad():
                y_pred = torch.argmax(model(test_data), dim=1)
                acc = accuracy_score(y_test.cpu().numpy(), y_pred.cpu().numpy())
                print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Accuracy: {acc:.4f}')

    # Evaluate on test set
    model.eval()
    with torch.no_grad():
        y_pred = torch.argmax(model(test_data), dim=1)
        test_acc = accuracy_score(y_test.cpu().numpy(), y_pred.cpu().numpy())
        precision = precision_score(y_test.cpu().numpy(), y_pred.cpu().numpy(), average='macro', zero_division=1)
        recall = recall_score(y_test.cpu().numpy(), y_pred.cpu().numpy(), average='macro')
        f1 = f1_score(y_test.cpu().numpy(), y_pred.cpu().numpy(), average='macro')

        accuracy_scores.append(test_acc)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

        print(f"Fold {fold} - Accuracy: {test_acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

# Print final results
print(f"\nTraining completed. Total time: {datetime.datetime.now() - start_time}")
print(f"Average accuracy: {np.mean(accuracy_scores):.4f} ± {np.std(accuracy_scores):.4f}")
print(f"Average precision: {np.mean(precision_scores):.4f} ± {np.std(precision_scores):.4f}")
print(f"Average recall: {np.mean(recall_scores):.4f} ± {np.std(recall_scores):.4f}")
print(f"Average F1 score: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")